In [16]:
import pandas as pd
import numpy as np

import praw

from pymongo import MongoClient

## Set Up MongoDB

In [3]:
client = MongoClient(port=12345) # this is the port set by the SSH tunnel
db = client.new_cool_db

#reddit_datascience_posts
#reddit_datascience_comments
        

In [4]:
# print collection names
db.collection_names()
# col = db.testData
# col

# # print each document
# for x in col.find():
#      print(x)

[]

## Connect to Reddit

In [19]:
reddit = praw.Reddit(client_id = '9p5TdvPWFQ2rLQ',
                     client_secret = 'dOvpA6FJO7UwkzbK6hRpbkX08gA',
                     user_agent = 'test')

## Some Reference Values

In [31]:
# Reference subreddit: datascience
# May expand to other subreddits, but let's stick to that one for now.
# If and when expansion occurrs, generate a list and iterate over it or something like that.

# Reference post: 'https://www.reddit.com/r/datascience/comments/6bo3mk/xkcd_machine_learning/dho6cmf/'

# Reference unix timestamps
# source: http://www.unixtimestamp.com/index.php
unix20110101 = 1293840000 #2011-01-01, 00:00:00
unix20120101 = 1325376000 #2012-01-01, 00:00:00
unix20130101 = 1356998400 #2013-01-01, 00:00:00
unix20140101 = 1388534400 #2014-01-01, 00:00:00
unix20150101 = 1420070400 #2015-01-01, 00:00:00
unix20160101 = 1451606400 #2016-01-01, 00:00:00
unix20170101 = 1483228800 #2017-01-01, 00:00:00

print('2011: ', (unix20120101 - unix20110101)/60/60/24)
print('2012: ', (unix20130101 - unix20120101)/60/60/24)
print('2013: ', (unix20140101 - unix20130101)/60/60/24)
print('2014: ', (unix20150101 - unix20140101)/60/60/24)
print('2015: ', (unix20160101 - unix20150101)/60/60/24)
print('2016: ', (unix20170101 - unix20160101)/60/60/24)

2011:  365.0
2012:  366.0
2013:  365.0
2014:  365.0
2015:  365.0
2016:  366.0


## Helpful Function Definition

In [22]:
def calc_end(start, days):
    daysecs = 24*60*60
    return start + daysecs*days

In [23]:
def get_stuff(subreddit, start, days, db_posts, db_comments, inPlace = False):

    comments = []
    post_info = []

    sub = reddit.subreddit(subreddit).submissions(start, calc_end(start, days))

    for post in sub:

        pi = {}
        pi['title'] = post.title
        pi['score']= post.score
        pi['id'] = post.id
        pi['selftext'] = post.selftext
        pi['created'] = post.created
        pi['name'] = str(post.author)

        cs = {}
        cs['post_id'] = post.id

        post.comments.replace_more(limit = 0)

        for i, comment in enumerate(post.comments.list()): #z, y
            comment_num = 'comment{}'.format(i)
            cs[comment_num] = comment.body

        if not inPlace:
            post_info.append(pi)
            comments.append(cs)

        db_posts.insert_one(pi)
        db_comments.insert_one(cs)

    if not inPlace:
        return comments, post_info
    else:
        return None

In [35]:
for num_days in np.append(np.linspace(1, 30, 5), [31]):
    sub = (reddit.subreddit('datascience')
           .search(create_timestamp(unix20160101, num_days), syntax = 'cloudsearch', limit = None))
    print(len(list(sub)))

2
40
82
120
164
167


## Obtain and Store Data